<a href="https://colab.research.google.com/github/edydkim/ezkl/blob/main/examples/notebooks/simple_demo_public_network_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## EZKL Jupyter Notebook Demo

Here we demonstrate how to use the EZKL package to run a publicly known / committed to network on some private data, producing a public output.


In [1]:
# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass


# here we create and (potentially train a model)

# make sure you have the dependencies required here already installed
from torch import nn
import ezkl
import os
import json
import torch


# Defines the model
# we got convs, we got relu, we got linear layers
# What else could one want ????

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=5, stride=2)
        self.conv2 = nn.Conv2d(in_channels=2, out_channels=3, kernel_size=5, stride=2)

        self.relu = nn.ReLU()

        self.d1 = nn.Linear(48, 48)
        self.d2 = nn.Linear(48, 10)

    def forward(self, x):
        # 32x1x28x28 => 32x32x26x26
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)

        # flatten => 32 x (32*26*26)
        x = x.flatten(start_dim = 1)

        # 32 x (32*26*26) => 32x128
        x = self.d1(x)
        x = self.relu(x)

        # logits => 32x10
        logits = self.d2(x)

        return logits


circuit = MyModel()

# Train the model as you like here (skipped for brevity)




In [2]:
model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')

witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')

In [3]:

shape = [1, 28, 28]
# After training, export to onnx (network.onnx) and create a data file (input.json)
x = 0.1*torch.rand(1,*shape, requires_grad=True)

# Flips the neural net into inference mode
circuit.eval()

    # Export the model
torch.onnx.export(circuit,               # model being run
                      x,                   # model input (or a tuple for multiple inputs)
                      model_path,            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=10,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})

data_array = ((x).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

    # Serialize data into file:
json.dump( data, open(data_path, 'w' ))


In [4]:
py_run_args = ezkl.PyRunArgs()
py_run_args.input_visibility = "private"
py_run_args.output_visibility = "public"
py_run_args.param_visibility = "fixed" # private by default

res = ezkl.gen_settings(model_path, settings_path, py_run_args=py_run_args)

assert res == True


In [5]:
cal_path = os.path.join("calibration.json")

data_array = (torch.rand(20, *shape, requires_grad=True).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

# Serialize data into file:
json.dump(data, open(cal_path, 'w'))


await ezkl.calibrate_settings(cal_path, model_path, settings_path, "resources")

ERROR:ezkl.graph.model:[tensor] decomposition error: integer -610150149 is too large to be represented by base 16384 and n 2
ERROR:ezkl.execute:forward pass failed: "failed to forward: [halo2] General synthesis error"
ERROR:ezkl.graph.model:[tensor] decomposition error: integer -2449049795 is too large to be represented by base 16384 and n 2
ERROR:ezkl.execute:forward pass failed: "failed to forward: [halo2] General synthesis error"
ERROR:ezkl.graph.model:[tensor] decomposition error: integer -9801518207 is too large to be represented by base 16384 and n 2
ERROR:ezkl.execute:forward pass failed: "failed to forward: [halo2] General synthesis error"
ERROR:ezkl.graph.model:[tensor] decomposition error: integer -4898872871 is too large to be represented by base 16384 and n 2
ERROR:ezkl.execute:forward pass failed: "failed to forward: [halo2] General synthesis error"
ERROR:ezkl.graph.model:[tensor] decomposition error: integer -19606117158 is too large to be represented by base 16384 and n 

True

In [6]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

In [7]:
# srs path
res = await ezkl.get_srs( settings_path)

In [8]:
# now generate the witness file

res = await ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

In [9]:

# HERE WE SETUP THE CIRCUIT PARAMS
# WE GOT KEYS
# WE GOT CIRCUIT PARAMETERS
# EVERYTHING ANYONE HAS EVER NEEDED FOR ZK



res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,

    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

In [10]:
# GENERATE A PROOF


proof_path = os.path.join('test.pf')

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,

        "single",
    )

print(res)
assert os.path.isfile(proof_path)

{'instances': [['b7fcffef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430', '1dfdffef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430', '2e03000000000000000000000000000000000000000000000000000000000000', '0600000000000000000000000000000000000000000000000000000000000000', '9cffffef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430', '9501000000000000000000000000000000000000000000000000000000000000', '1afbffef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430', '7c00000000000000000000000000000000000000000000000000000000000000', 'a9fcffef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430', 'cdfbffef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430']], 'proof': '0x0decd8aae0a325fcae52fdad3f625da5b99bc113c3e17a48bd1e4870e573541a177fe247fb44156a288bcdda5a0ca58eb9f2c0abe021b8628c0d4af352091d7a15cd7235f9a4d9681c8f67c9ad84464a351f6ef0ee8637559bbbc44f2223bdbf2b9a8bcaf136237a12522f3c805dd30a9cb0af7dee5e4232a25dbf28a782e16d0086f2a1b19fa5df81c4523e5f74d58770

In [11]:
# VERIFY IT

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,

    )

assert res == True
print("verified")

verified
